In [ ]:
!pip install progressbar2

In [ ]:
import time
import progressbar
import pandas as pd
import numpy as np
from osgeo import ogr
from numba import jit, cuda 

In [ ]:
!unzip /content/drive/MyDrive/WaterPollution/finalData.zip

Archive:  /content/drive/MyDrive/WaterPollution/finalData.zip
  inflating: finalData.csv           


In [ ]:
MAIN_PATH = "/content/finalData.csv"
POPULATION_PATH = "/content/drive/MyDrive/WaterPollution/population_2018.csv"
PROTECTEDMARINES_PATH = "/content/drive/MyDrive/WaterPollution/protectedMarinesData.csv"
TOURISTDATA = "/content/drive/MyDrive/WaterPollution/touristData.csv"
MIGRATION_DATA = "/content/drive/MyDrive/WaterPollution/migrationData.csv"
CLIMATE_DATA = "/content/drive/MyDrive/WaterPollution/climateData.csv"
LITERACY_DATA = "/content/drive/MyDrive/WaterPollution/literacyRate.csv"
RENEWABLE_DATA = "/content/drive/MyDrive/WaterPollution/renewable.csv"
WASTE_DATA = "/content/drive/MyDrive/WaterPollution/wasteData.csv"
DATA100K_PATH = "/content/drive/MyDrive/WaterPollution/data100k.csv" 

In [ ]:
main_df = pd.read_csv(MAIN_PATH)
populationData = pd.read_csv(POPULATION_PATH)
protectedMarinesData = pd.read_csv(PROTECTEDMARINES_PATH)
touristData = pd.read_csv(TOURISTDATA)
migrationData = pd.read_csv(MIGRATION_DATA)
climateData = pd.read_csv(CLIMATE_DATA)
literacyData = pd.read_csv(LITERACY_DATA)
renewableData = pd.read_csv(RENEWABLE_DATA)
wasteData = pd.read_csv(WASTE_DATA)

In [ ]:
a = pd.DataFrame((main_df.isna().sum() / main_df.shape[0])  * 100,columns=["naPercentage"]).reset_index()
a

,index,naPercentage
0,monitoringSiteIdentifier,0.000000
1,monitoringSiteIdentifierScheme_x,0.000000
2,parameterWaterBodyCategory,0.000000
3,observedPropertyDeterminandCode,0.000000
4,procedureAnalysedFraction,0.000000
5,procedureAnalysedMedia,0.000000
6,resultUom,0.000000
7,phenomenonTimeReferenceYear,0.000000
8,parameterSamplingPeriod,1.002799
9,procedureLOQValue,50.268637


In [ ]:
def cleanData(main_df):
  main_df.drop(["procedureLOQValue","resultQualityNumberOfSamplesBelowLOQ","resultQualityMinimumBelowLOQ",
                "resultQualityMinimumBelowLOQ","resultQualityMeanBelowLOQ","resultQualityMedianBelowLOQ",
                "resultMedianValue","resultStandardDeviationValue","procedureAnalyticalMethod","parameterSampleDepth",
                "resultObservationStatus","Remarks","metadata_statements","lon_x","lat_x","lon_y","lat_y","resultQualityMaximumBelowLOQ"],axis=1,
              inplace = True)
  main_df.dropna(axis=0,inplace=True)
  main_df.drop(["monitoringSiteIdentifier","resultNumberOfSamples","resultMinimumValue","resultMaximumValue","metadata_versionId","metadata_beginLifeSpanVersion","metadata_statusCode",
              "metadata_observationStatus","UID","monitoringSiteIdentifierScheme_y","waterBodyIdentifierScheme","confidentialityStatus"],axis=1,inplace = True)

In [ ]:
cleanData(main_df)

In [ ]:
main_df.head()

,monitoringSiteIdentifierScheme_x,parameterWaterBodyCategory,observedPropertyDeterminandCode,procedureAnalysedFraction,procedureAnalysedMedia,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,resultMeanValue,waterBodyIdentifier,lon,lat
0,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.063310,FRDR609,4.99635,46.98310
1,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.046733,FRFR238,-0.23809,43.67623
2,euMonitoringSiteCode,RW,EEA_3164-07-6,total,water,{massRatio},2009,2009-01--2009-12,132.859000,FRFR238,-0.23809,43.67623
3,euMonitoringSiteCode,RW,CAS_14797-55-8,total,water,mg{NO3}/L,2009,2009-01--2009-12,11.578376,FRFR327C,-0.15118,43.65393
4,eionetMonitoringSiteCode,RW,EEA_3151-01-7,total,water,mmol/L,2009,2009-01--2009-12,0.206800,ES014MSPFES-014-NR-038-000-02-02,-7.59115,43.61870


# Concat Data by Country

In [ ]:
class Point(object):
    def __init__(self, lat, lng):
        self.point = ogr.Geometry(ogr.wkbPoint)
        self.point.AddPoint(lng, lat)
    
    def getOgr(self):
        return self.point
    ogr = property(getOgr)

class Country(object):
    def __init__(self, shape):
        self.shape = shape
    
    def getIso(self):
        return self.shape.GetField('ISO2')
    iso = property(getIso)
    
    def __str__(self):
        return self.shape.GetField('NAME')
    
    def contains(self, point):
        return self.shape.geometry().Contains(point.ogr)

class CountryChecker(object):
    
    def __init__(self, country_file):
        driver = ogr.GetDriverByName('ESRI Shapefile')
        self.countryFile = driver.Open(country_file)
        self.layer = self.countryFile.GetLayer()
    
    def getCountry(self, point):
        
        for i in range(self.layer.GetFeatureCount()):
            country = self.layer.GetFeature(i)
            if country.geometry().Contains(point.ogr):
                return Country(country)       
        return None

In [ ]:
!wget http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip
!unzip /content/TM_WORLD_BORDERS-0.3.zip

--2021-01-04 13:34:05--  http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip
Resolving thematicmapping.org (thematicmapping.org)... 67.20.120.230
Connecting to thematicmapping.org (thematicmapping.org)|67.20.120.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3464707 (3.3M) [application/zip]
Saving to: ‘TM_WORLD_BORDERS-0.3.zip’

TM_WORLD_BORDERS-0. 100%[===================>]   3.30M  6.50MB/s    in 0.5s    

2021-01-04 13:34:06 (6.50 MB/s) - ‘TM_WORLD_BORDERS-0.3.zip’ saved [3464707/3464707]

Archive:  /content/TM_WORLD_BORDERS-0.3.zip
  inflating: Readme.txt              
  inflating: TM_WORLD_BORDERS-0.3.dbf  
  inflating: TM_WORLD_BORDERS-0.3.prj  
  inflating: TM_WORLD_BORDERS-0.3.shp  
  inflating: TM_WORLD_BORDERS-0.3.shx  


In [ ]:
def getCountry(lat,lon,countryChecker = CountryChecker("/content/TM_WORLD_BORDERS-0.3.shp")):
  return str(countryChecker.getCountry(Point(lat,lon)))

In [ ]:
@jit
def fillCountry(main_df):
  main_df["Country"] = 0
  for i in progressbar.progressbar(range(main_df.shape[0])):
      lat, lon = main_df["lat"][i], main_df["lon"][i]
      main_df["Country"][i] = getCountry(lat,lon)

In [ ]:
main_df.reset_index(inplace= True)
new_data1 = main_df[:100000]

In [ ]:
fillCountry(new_data1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
N/A% (0 of 100000) |                     | Elapsed Time: 0:00:00 ETA:  --:--:--/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveat

In [ ]:
new_data1["Country"].unique()

array(['France', 'Spain', 'United Kingdom', 'Lithuania', 'Portugal',
       'Austria', 'Bulgaria', 'Germany', 'Finland', 'Czech Republic',
       'Belgium', 'Denmark', 'None', 'Italy', 'Ireland', 'Sweden',
       'Romania', 'Serbia', 'Slovakia', 'Netherlands', 'Poland',
       'Luxembourg', 'Switzerland', 'Latvia', 'Ukraine', 'Norway',
       'Croatia', 'Belarus', 'Russia', 'Republic of Moldova', 'Slovenia',
       'Greece', 'Bosnia and Herzegovina', 'Hungary'], dtype=object)

In [ ]:
new_data1.to_csv("countryCsv.csv",index=False)

In [ ]:
print(5)

5


In [ ]:
populationData.head()

,Country Name,Country Code,2018
0,Aruba,ABW,588.027778
1,Afghanistan,AFG,56.937760
2,Angola,AGO,24.713052
3,Albania,ALB,104.612263
4,Andorra,AND,163.842553


In [ ]:
datas = (populationData,protectedMarinesData,touristData,migrationData,climateData,literacyData,renewableData)

In [ ]:
for i in datas:
  i.rename(columns = {"Country Name":"Country"},inplace=True)
wasteData.rename(columns={"country_name":"Country"},inplace=True)

In [ ]:
touristData.head()

,Country,Country Code,TouristMean_1990_2020
0,Aruba,ABW,755057
1,Afghanistan,AFG,0
2,Angola,AGO,231346
3,Albania,ALB,1379653
4,Andorra,AND,1975269


In [ ]:
population = pd.merge(new_data1,populationData,how="left",on = ["Country"])
population.rename(columns = {"2018":"PopulationDensity"},inplace=True)

In [ ]:
marines = pd.merge(population,protectedMarinesData,how="left",on = ["Country"])

In [ ]:
allData = pd.merge(marines,touristData,how="left",on = ["Country"])
allData.head()

,level_0,index,monitoringSiteIdentifierScheme_x,parameterWaterBodyCategory,observedPropertyDeterminandCode,procedureAnalysedFraction,procedureAnalysedMedia,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,resultMeanValue,waterBodyIdentifier,lon,lat,Country,Country Code_x,PopulationDensity,Country Code_y,TerraMarineProtected_2016_2018,Country Code,TouristMean_1990_2020
0,0,0,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.063310,FRDR609,4.99635,46.98310,France,FRA,122.299437,FRA,30.831906,FRA,71176346.0
1,1,1,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.046733,FRFR238,-0.23809,43.67623,France,FRA,122.299437,FRA,30.831906,FRA,71176346.0
2,2,2,euMonitoringSiteCode,RW,EEA_3164-07-6,total,water,{massRatio},2009,2009-01--2009-12,132.859000,FRFR238,-0.23809,43.67623,France,FRA,122.299437,FRA,30.831906,FRA,71176346.0
3,3,3,euMonitoringSiteCode,RW,CAS_14797-55-8,total,water,mg{NO3}/L,2009,2009-01--2009-12,11.578376,FRFR327C,-0.15118,43.65393,France,FRA,122.299437,FRA,30.831906,FRA,71176346.0
4,4,4,eionetMonitoringSiteCode,RW,EEA_3151-01-7,total,water,mmol/L,2009,2009-01--2009-12,0.206800,ES014MSPFES-014-NR-038-000-02-02,-7.59115,43.61870,Spain,ESP,93.677197,ESP,15.047884,ESP,50941692.0


In [ ]:
allData = pd.merge(allData,migrationData,how="left",on = ["Country"])
allData = pd.merge(allData,climateData,how="left",on = ["Country"])
allData = pd.merge(allData,literacyData,how="left",on = ["Country"])
allData = pd.merge(allData,renewableData,how="left",on = ["Country"])
allData = pd.merge(allData,wasteData,how="left",on = ["Country"])

In [ ]:
allData.columns

Index(['level_0', 'index', 'monitoringSiteIdentifierScheme_x',
       'parameterWaterBodyCategory', 'observedPropertyDeterminandCode',
       'procedureAnalysedFraction', 'procedureAnalysedMedia', 'resultUom',
       'phenomenonTimeReferenceYear', 'parameterSamplingPeriod',
       'resultMeanValue', 'waterBodyIdentifier', 'lon', 'lat', 'Country',
       'Country Code_x', 'PopulationDensity', 'Country Code_y',
       'TerraMarineProtected_2016_2018', 'Country Code',
       'TouristMean_1990_2020'],
      dtype='object')

In [ ]:
allData.drop(["Country Code_x","Country Code_y","Country Code","level_0","index"],axis = 1 ,inplace = True)

In [ ]:
allData.to_csv("data100k.csv",index=False)

# VENUES

In [ ]:
newData = pd.read_csv(DATA100K_PATH)

In [ ]:
from pandas.io.json import json_normalize
import pandas as pd
import requests

class FoursquareAPI():
	def __init__(self,lat, lon, limit = 100, radius = 500):
		self.lat = lat
		self.lon = lon
		self.limit = limit
		self.radius = radius
		self.clientID = "B3UDNS1NEH0N0J3SOUDKQM3GKQWSXLBCC33TXEI51KGSZJCP"
		self.clientSecret = "LTUFQKVIX1GRWZWXN31E5TLAIMJKFAOX14S0MMUANGQBGI3V"
		self.version = "20180604"
		self.queryResults = pd.DataFrame()
		self.status = 0

	def setParams(self,lat = None,lon = None,limit = None,radius = None):
		if lat != None:
			self.lat = lat
		if lon != None:
			self.lon = lon
		if limit != None:
			self.limit = limit
		if radius != None:
			self.radius = radius

	def get_category_type(self,row):
		try:
			categories_list = row['categories']
		except:
			categories_list = row['venue.categories']
			
		if len(categories_list) == 0:
			return None
		else:
			return categories_list[0]['name']

	def runQuery(self):
		url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(self.clientID, self.clientSecret, self.version, self.lat,self.lon,self.radius,self.limit)
		results = requests.get(url).json()
		try:
			venues = results['response']['groups'][0]['items']

              #flatten JSON, Normalize JSON to Dataframe
			dataframe = json_normalize(venues) 

              # filter columns,consider only required columns
              
			filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
			self.queryResults = dataframe.loc[:, filtered_columns]

                # filter the category for each row
			self.queryResults['venue.categories'] = self.queryResults.apply(self.get_category_type, axis=1)

                # clean columns
			self.queryResults.columns = [col.split('.')[-1] for col in self.queryResults.columns]
			self.status = 2
		except:
			self.status = 1

	def getVenuesTable(self):
		if self.status == 0:
			raise Exception("Run the query first. queryResults table is empty.")
		elif self.status == 1:
			raise Exception(f"Table is empty,there are not any venues in {self.radius} radius")
		else:
			return self.queryResults

	def getVenuesCount(self):
		if self.status == 0:
			raise Exception("Run the query first. queryResults table is empty.")		
		elif self.status == 1:
			return 0
		else:
			return self.queryResults.shape[0]
			

	def getParams(self):
		params = {"Latitude": self.lat,"Longtitude" : self.lon, "Radius": self.radius, "Place_limit": self.limit,
		"Client_id" : self.clientID, "Client_secret": self.clientSecret,"Version":self.version}
		return params


In [ ]:
subData = newData[:20000]

In [ ]:
def getVenueCount(main_df):
  main_df["VenueCount"] = np.nan
  api = FoursquareAPI(0,0,radius=1000)
  for i in progressbar.progressbar(range(main_df.shape[0])):
      api.setParams(lat = main_df["lat"][i],lon = main_df["lon"][i])
      api.runQuery()
      main_df["VenueCount"][i] = api.getVenuesCount()

In [ ]:
getVenueCount(subData)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
N/A% (0 of 20000) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is

In [ ]:
subData.to_csv("dataWithVenues.csv",index = False)

#OSMNX

In [ ]:
!apt install python3-rtree
!pip install osmnx==0.16.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources
Suggested packages:
  python3-setuptools
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
  python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-pkg-resources all 39.0.1-2 [98.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 k

In [ ]:
subData = pd.read_csv("dataWithVenues.csv")

In [ ]:
subData.head()

,monitoringSiteIdentifierScheme_x,parameterWaterBodyCategory,observedPropertyDeterminandCode,procedureAnalysedFraction,procedureAnalysedMedia,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,resultMeanValue,waterBodyIdentifier,lon,lat,Country,PopulationDensity,TerraMarineProtected_2016_2018,TouristMean_1990_2020,VenueCount
0,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.063310,FRDR609,4.99635,46.98310,France,122.299437,30.831906,71176346.0,0.0
1,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.046733,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0
2,euMonitoringSiteCode,RW,EEA_3164-07-6,total,water,{massRatio},2009,2009-01--2009-12,132.859000,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0
3,euMonitoringSiteCode,RW,CAS_14797-55-8,total,water,mg{NO3}/L,2009,2009-01--2009-12,11.578376,FRFR327C,-0.15118,43.65393,France,122.299437,30.831906,71176346.0,2.0
4,eionetMonitoringSiteCode,RW,EEA_3151-01-7,total,water,mmol/L,2009,2009-01--2009-12,0.206800,ES014MSPFES-014-NR-038-000-02-02,-7.59115,43.61870,Spain,93.677197,15.047884,50941692.0,4.0


In [ ]:
import osmnx as ox
import networkx as nx
import os
import matplotlib.colors as colors

In [ ]:
def get_distance(data, feature, road):
  if len(feature) == 0:
    lat = data['lat']
    lon = data['lon']
  else:
    lat = data.loc[feature,'lat']
    lon = data.loc[feature,'lon']
  dist = [ox.distance.get_nearest_edge(road, point=(x, y), return_dist=True) for x, y in zip(lat, lon)]
  return [x[3]*111139 for x in dist]

# Dummy

In [ ]:
@jit(nopython=True)
def findNearestRoad(coords):
  G = ox.graph_from_point(coords, dist=500, network_type='none',custom_filter='["highway"]')
  dist = ox.distance.get_nearest_edge(G, point=coords, return_dist=True)
  return dist[3]*111139

In [ ]:
@jit(nopython=True)
def getRoads(latList,lonList):
  deneme = np.zeros_like(latList)
  for i in range(latList.shape[0]):
      lat,lon =latList[i], lonList[i]
      deneme[i] = findNearestRoad((lat,lon))
  return deneme

In [ ]:
venues = pd.read_csv("/content/dataWithVenues.csv")

In [ ]:
venues.head()

,monitoringSiteIdentifierScheme_x,parameterWaterBodyCategory,observedPropertyDeterminandCode,procedureAnalysedFraction,procedureAnalysedMedia,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,resultMeanValue,waterBodyIdentifier,lon,lat,Country,PopulationDensity,TerraMarineProtected_2016_2018,TouristMean_1990_2020,VenueCount
0,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.063310,FRDR609,4.99635,46.98310,France,122.299437,30.831906,71176346.0,0.0
1,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.046733,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0
2,euMonitoringSiteCode,RW,EEA_3164-07-6,total,water,{massRatio},2009,2009-01--2009-12,132.859000,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0
3,euMonitoringSiteCode,RW,CAS_14797-55-8,total,water,mg{NO3}/L,2009,2009-01--2009-12,11.578376,FRFR327C,-0.15118,43.65393,France,122.299437,30.831906,71176346.0,2.0
4,eionetMonitoringSiteCode,RW,EEA_3151-01-7,total,water,mmol/L,2009,2009-01--2009-12,0.206800,ES014MSPFES-014-NR-038-000-02-02,-7.59115,43.61870,Spain,93.677197,15.047884,50941692.0,4.0


In [ ]:
wasteData.rename(columns={"country_name":"Country"},inplace=True)
venues = pd.merge(venues,migrationData,how="left",on = ["Country"])
venues = pd.merge(venues,climateData,how="left",on = ["Country"])
venues = pd.merge(venues,literacyData,how="left",on = ["Country"])
venues = pd.merge(venues,renewableData,how="left",on = ["Country"])
venues = pd.merge(venues,wasteData,how="left",on = ["Country"])

In [ ]:
venues.head()

,monitoringSiteIdentifierScheme_x,parameterWaterBodyCategory,observedPropertyDeterminandCode,procedureAnalysedFraction,procedureAnalysedMedia,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,resultMeanValue,waterBodyIdentifier,lon,lat,Country,PopulationDensity,TerraMarineProtected_2016_2018,TouristMean_1990_2020,VenueCount,netMigration_2011_2018,droughts_floods_temperature,literacyRate_2010_2018,combustibleRenewables_2009_2014,gdp,composition_food_organic_waste_percent,composition_glass_percent,composition_metal_percent,composition_other_percent,composition_paper_cardboard_percent,composition_plastic_percent,composition_rubber_leather_percent,composition_wood_percent,composition_yard_garden_green_waste_percent,waste_treatment_recycling_percent
0,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.063310,FRDR609,4.99635,46.98310,France,122.299437,30.831906,71176346.0,0.0,75808.375,0.005718,0.000000,4.457840,2.806036e+12,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,0.0,22.26
1,euMonitoringSiteCode,RW,CAS_14797-65-0,total,water,mg{NO2}/L,2009,2009-01--2009-12,0.046733,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0,75808.375,0.005718,0.000000,4.457840,2.806036e+12,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,0.0,22.26
2,euMonitoringSiteCode,RW,EEA_3164-07-6,total,water,{massRatio},2009,2009-01--2009-12,132.859000,FRFR238,-0.23809,43.67623,France,122.299437,30.831906,71176346.0,0.0,75808.375,0.005718,0.000000,4.457840,2.806036e+12,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,0.0,22.26
3,euMonitoringSiteCode,RW,CAS_14797-55-8,total,water,mg{NO3}/L,2009,2009-01--2009-12,11.578376,FRFR327C,-0.15118,43.65393,France,122.299437,30.831906,71176346.0,2.0,75808.375,0.005718,0.000000,4.457840,2.806036e+12,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,0.0,22.26
4,eionetMonitoringSiteCode,RW,EEA_3151-01-7,total,water,mmol/L,2009,2009-01--2009-12,0.206800,ES014MSPFES-014-NR-038-000-02-02,-7.59115,43.61870,Spain,93.677197,15.047884,50941692.0,4.0,-40055.250,0.729194,87.158924,4.363288,1.464509e+12,49.0,8.0,3.0,14.0,15.0,9.0,0.0,2.0,0.0,16.84


In [ ]:
venues.drop(["monitoringSiteIdentifierScheme_x","lon","lat"],axis = 1,inplace=True)
venues.to_csv("waterPollution.csv",index=False)